In [1]:
# !pip install duckdb

In [2]:
import pandas as pd
import duckdb

# Using DuckDB

We didn't introduce DuckDB in the book. DuckDB is an in-process SQL **OLAP** database management system designed for fast **analytical queries** on **large datasets**, often used for **data/business analysis and embedded applications**.

In this small notebook, we show a couple of use cases of DuckDB, we use the parquet file called `yellow_tripdata_2024-01.parquet` from [NYC's TLC trip data](https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page).We don't really mind cleaning the data here, we just want to show how to use DuckDB if you have never used it. We really hope that you'll [check their documentation out](https://duckdb.org/docs/clients/python/overview.html) because it's a fantastic project, **and you can toally leverage it in your Taipy applications!**

This notebook shows how to use DuckDB, your goal is to bring it to a Taipy application: let users construct queries with visual elements!

## Check Data with pandas

In [3]:
df = pd.read_parquet("./data/yellow_tripdata_2024-01.parquet")

In [4]:
df.head(2)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,2,2024-01-01 00:57:55,2024-01-01 01:17:43,1.0,1.72,1.0,N,186,79,2,17.7,1.0,0.5,0.00,0.0,1.0,22.70,2.5,0.0
1,1,2024-01-01 00:03:00,2024-01-01 00:09:36,1.0,1.80,1.0,N,140,236,1,10.0,3.5,0.5,3.75,0.0,1.0,18.75,2.5,0.0


In [5]:
df.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'Airport_fee'],
      dtype='object')

## Let's use DuckDB

DuckDB lets you create SQL queries from parquet files (among many other things!).

Here is a process:
1- Create a connection with `conn = duckdb.connect()`
2- Create a SQL query, like you would for a database table. 
3- Execute the query, and fetch it (use `conn.execute(query).fetchdf()`). The method `fetchdf` returns a pandas DataFrame.


In [6]:
# Connect to DuckDB
conn = duckdb.connect()

# Read the Parquet file and calculate the average tip and the total tips for January, briken by passenger_count
query = """
SELECT
    COALESCE(passenger_count, 0) AS passenger_count, -- COALESCE replaces Null values
    COUNT(*) AS number_trips,
    SUM(tip_amount) AS total_tips,
    AVG(tip_amount) AS average_tip,
FROM 
    './data/yellow_tripdata_2024-01.parquet'
WHERE 
    passenger_count > 0
    AND payment_type = 1
GROUP BY ALL
ORDER BY average_tip DESC
"""

# Execute the query and fetch the result as a pandas dataframe
df_payment_type = conn.execute(query).fetchdf()

# Close the DuckDB connection
conn.close()

df_payment_type

,passenger_count,number_trips,total_tips,average_tip
0,8,46,6.105800e+02,13.273478
1,7,7,6.696000e+01,9.565714
2,4,37588,1.800310e+05,4.789587
3,2,325715,1.505162e+06,4.621103
4,3,71888,3.226025e+05,4.487571
5,5,27650,1.132405e+05,4.095497
6,1,1812609,7.376525e+06,4.069562
7,6,18502,7.475064e+04,4.040138
8,9,1,3.050000e+00,3.050000


## A single-value select

In [7]:
# Connect to DuckDB
conn = duckdb.connect()

# Read the Parquet file and calculate the average tip and the total tips for January, briken by passenger_count
query = """
SELECT
    MAX(tip_amount) AS max_tip
FROM 
    './data/yellow_tripdata_2024-01.parquet'
WHERE 
    payment_type = 1
"""

# Execute the query and fetch the result
df_payment_type = conn.execute(query).fetchdf()


# Close the DuckDB connection
conn.close()

df_payment_type

,max_tip
0,428.0


## Another example:

In [8]:
conn = duckdb.connect()

query = """
SELECT
    CASE
        WHEN trip_distance < 1 THEN 'Less than 1'
        WHEN trip_distance BETWEEN 1 AND 2 THEN '1 to 2'
        WHEN trip_distance BETWEEN 2 AND 3 THEN '2 to 3'
        WHEN trip_distance BETWEEN 3 AND 4 THEN '3 to 4'
        WHEN trip_distance BETWEEN 4 AND 5 THEN '4 to 5'
        WHEN trip_distance BETWEEN 5 AND 10 THEN '5 to 10'
        WHEN trip_distance BETWEEN 10 AND 20 THEN '10 to 20'
        ELSE 'More than 20'
    END AS trip_distance_range,
    COUNT(*) AS trip_count,
    SUM(tip_amount) AS total_tips,
    AVG(tip_amount) AS average_tip,
FROM './data/yellow_tripdata_2024-01.parquet'
WHERE payment_type = 1
GROUP BY trip_distance_range
ORDER BY average_tip DESC;
"""

df_stats = conn.execute(query).fetchdf()

conn.close()

df_stats

,trip_distance_range,trip_count,total_tips,average_tip
0,More than 20,22878,350852.58,15.335806
1,10 to 20,157080,1859936.73,11.840697
2,5 to 10,176708,1279596.95,7.241307
3,4 to 5,80286,413355.94,5.148543
4,3 to 4,161495,727632.95,4.505607
5,2 to 3,362446,1355601.08,3.740146
6,1 to 2,814209,2372273.42,2.913593
7,Less than 1,543944,1310408.34,2.409087


In [9]:
type(df_stats)

pandas.core.frame.DataFrame

## One last example: Use Common Table Expresions (CTEs)

In [10]:
conn = duckdb.connect()

query = """
WITH cte_tip_per_mile AS (
    SELECT
        trip_distance,
        tip_amount,
        tip_amount / trip_distance AS tip_per_mile,
        payment_type
    FROM './data/yellow_tripdata_2024-01.parquet'
    WHERE trip_distance > 0
)
SELECT
    CASE
        WHEN trip_distance < 1 THEN 'Less than 1'
        WHEN trip_distance BETWEEN 1 AND 2 THEN '1 to 2'
        WHEN trip_distance BETWEEN 2 AND 3 THEN '2 to 3'
        WHEN trip_distance BETWEEN 3 AND 4 THEN '3 to 4'
        WHEN trip_distance BETWEEN 4 AND 5 THEN '4 to 5'
        WHEN trip_distance BETWEEN 5 AND 10 THEN '5 to 10'
        WHEN trip_distance BETWEEN 10 AND 20 THEN '10 to 20'
        ELSE 'More than 20'
    END AS trip_distance_range,
    COUNT(*) AS trip_count,
    SUM(tip_amount) AS total_tips,
    AVG(tip_amount) AS average_tip,
    AVG(tip_per_mile) AS average_tip_per_mile
FROM cte_tip_per_mile
WHERE payment_type = 1
GROUP BY trip_distance_range
ORDER BY average_tip_per_mile DESC;
"""

stats_df = conn.execute(query).fetchdf()

conn.close()

stats_df

,trip_distance_range,trip_count,total_tips,average_tip,average_tip_per_mile
0,Less than 1,523340,1190872.88,2.275524,5.261360
1,1 to 2,814209,2372273.42,2.913593,2.063195
2,2 to 3,362446,1355601.08,3.740146,1.539856
3,3 to 4,161495,727632.95,4.505607,1.308883
4,4 to 5,80286,413355.94,5.148543,1.154865
5,5 to 10,176708,1279596.95,7.241307,0.988823
6,10 to 20,157080,1859936.73,11.840697,0.806249
7,More than 20,22878,350852.58,15.335806,0.647055
